In [1]:
from __future__ import print_function
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.layers import Input
from keras.models import Model
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from keras.optimizers import Adam
from keras.constraints import maxnorm


Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense, GRU

In [3]:
def data():
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    # if data set not in the same folder as this code then add the complete path of the data to pandas.read_csv()
    X = pd.read_csv("data_multi_final.csv")
    Y= X[["beta", "gamma"]]
    X = X.iloc[:, 1:19]
    X.drop(["X1.10000"], axis=1, inplace=True)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=8000, test_size=2000, random_state=4)
    X_train = np.array(X_train)
    X_train = X_train[:, :, np.newaxis]
    X_test = np.array(X_test)
    X_test = X_test[:, :, np.newaxis]
    return X_train, Y_train, X_test, Y_test

In [4]:
def model(X_train, Y_train, X_test, Y_test):
    main_input = Input(shape=(17,1), name='main_input')
    lstm_out = GRU({{choice([50, 100, 200, 300])}}, dropout={{choice([0, 0.1, 0.2, 0.3])}}, recurrent_dropout={{choice([0, 0.1, 0.2, 0.3])}})(main_input)
    out1 = Dense(1,  activation='linear', name='out1')(lstm_out)
    out2 = Dense(1,  activation='linear',name='out2')(lstm_out)
    model = Model(inputs=main_input, outputs=[out1,out2])
    adam = keras.optimizers.Adam(lr={{choice([10**-3, 10**-2, 10**-4, 10**-5])}})
    model.compile(optimizer = adam,
                   loss={'out1':'mean_squared_error', 'out2':'mean_squared_error'})
    model.fit({'main_input':X_train}, {'out1':Y_train['beta'], 'out2':Y_train['gamma']},
               nb_epoch={{choice([100, 200])}},
               batch_size={{choice([10, 20,30, 40])}},
               verbose=2,
               validation_data=(X_test, [Y_test['beta'], Y_test['gamma']]))

    scores = model.evaluate(X_test,[Y_test['beta'], Y_test['gamma']])
    return {'loss': scores[0], 'status': STATUS_OK, 'model': model}

In [1]:
X_train, Y_train, X_test, Y_test = data()

best_run_lstm, best_model_lstm = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=31,
                                      trials=Trials(),
                                      notebook_name='HyperasGRU')

In [7]:
best_run_lstm

{'GRU': 1,
 'batch_size': 0,
 'dropout': 0,
 'dropout_1': 1,
 'lr': 2,
 'nb_epoch': 1}

In [8]:
best_model_lstm.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 17, 1)        0                                            
__________________________________________________________________________________________________
gru_31 (GRU)                    (None, 100)          30600       main_input[0][0]                 
__________________________________________________________________________________________________
out1 (Dense)                    (None, 1)            101         gru_31[0][0]                     
__________________________________________________________________________________________________
out2 (Dense)                    (None, 1)            101         gru_31[0][0]                     
Total params: 30,802
Trainable params: 30,802
Non-trainable params: 0
_______________________________________